# **Training à l'aide de la plateforme Detectron2**

**Detectron2** est une plateforme de recherche et detection d'objet crée par Facebook. Elle est codée sur Pytorch, et permet donc d'utiliser un GPU et d'être plus rapide. Je l'utilise pour implémenter un Faster R-CNN. Il pourrait être interessant d'explorer d'autres modeles de détection, comme des  Mask R-CNN.

[**Documentation de Detectron**](https://detectron2.readthedocs.io/en/latest/index.html)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

# Preprocessing

## Installations

In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
# Installer detectron2
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

## Imports et Dossiers



In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow
import json

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

In [ ]:
# Data est le dosssier mère contenant les sous dossiers test et train
Data = "drive/MyDrive/Mines Nancy/Depinfo/Projet/Faster_rCNN/Data_detectron"

# Dossiers de train et de test créés pendant le Preprocessing
train_path = Data + "/train"
test_path = Data + "/test"

COLORS = ['b', 'r', 'm', 'y', 'w', 'k']
category_id = {"smoke" : "0"}

## Enregistrement du Dataset



Il y a différents types de format pour les bbox. J'ai utilisé le formax **BoxMode.XYWH_ABS**, c'est à dire x0, y0, largeur, hauteur. Il faut enregistrer le dataset pour utiliser Detectron, dans leur DatasetCatalog, ainsi que les meta données dans MetadataCatalog (voir la [documentation](https://detectron2.readthedocs.io/en/latest/index.html)).


In [ ]:
def get_dicts(imgdir):
    json_file = imgdir+"/dataset_total.json"
    with open(json_file) as f:
        dataset_dicts = json.load(f)
    for i in dataset_dicts:
        filename = i["file_name"] 
        i["file_name"] = imgdir+"/"+filename 
        for j in i["annotations"]:
            j["bbox_mode"] = BoxMode.XYWH_ABS
            j["category_id"] = int(j["category_id"])
    return dataset_dicts

# Remplacer train et test par les noms des dossiers de train et test si ils ne s'appelent pas ainsi
# Il faudra modifier également data_dict_train et data_dict_test pour changer le dernier mot en le nom des dossiers dans le reste du notebook
for d in ["train", "test"]:
    DatasetCatalog.register("data_dict_" + d, lambda d=d: get_dicts(Data + "/" + d))
    MetadataCatalog.get("data_dict_" + d).set(thing_classes=["smoke"])
metadata_dict = MetadataCatalog.get("data_dict_train")

## Visualiser le Dataset

Code pour visualiser les images avec le Visualizer. Ici, on visualise 10 images de training.

In [ ]:
dataset_dicts = get_dicts(train_path)

for i in range(10):
    d = dataset_dicts[i]
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata_dict)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

# Training

In [ ]:
# On importe un module d'evaluation, en utilisant la validation de COCO
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs(Data + "/coco_eval_4", exist_ok=True)
        output_folder = Data + "/coco_eval_4"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

Configuration des paramètres et hyperparamètres.
J'utilise le modele **"faster_rcnn_R_50_FPN_3x.yaml"**.

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_dict_train",)
cfg.DATASETS.TEST = ("data_dict_test",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4

cfg.SOLVER.BASE_LR = 0.0125 
cfg.SOLVER.MAX_ITER = 3000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256  

# Nombre de classes, ici 1
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

cfg.TEST.EVAL_PERIOD = 500
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)

#  Lance l'entrainement
trainer.train()

Permet d'analyser comment l'entrainement s'est passé

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

# Analyse

## Inference sur le Dataset de Test avec le modèle entrainé

Un dossier output est sauvegardé dans le stokage local de google colab avec les poids du modèle entrainé. Il faut le sauvegarder pour réutiliser le modèle. 

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

# Permet de fixer le score minimal d'acceptation d'une image
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
cfg.DATASETS.TEST = ("data_dict_test", )
predictor = DefaultPredictor(cfg)


In [ ]:
from shutil import copytree
# On copie le dossier d'output contenant les poids au dossier dest. 
src = cfg.OUTPUT_DIR
dest = "drive/MyDrive/Mines Nancy/Depinfo/Projet/Faster_rCNN/Data_detectron/trained_model"
destination = copytree(src, dest)
print("Les poids ont été copié vers : ", destination)

In [ ]:
# Affiche les resultats sur des images aléatoires

from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_dicts(test_path)
for d in random.sample(dataset_dicts, 10):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata_dict, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE 
    )

    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

## Evaluation du modèle

Evaluation selon les critères COCO, avec le mAP. 

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_dict_test", cfg, False, output_dir="/output/")
val_loader = build_detection_test_loader(cfg, "data_dict_test")
inference_on_dataset(predictor.model, val_loader, evaluator)

## Codes en cours

In [ ]:

# Affiche les bboxs détectées sur les images de test
import detectron2.structures as structures
dataset_dicts = get_dicts(test_path)
for i in range(10) :
    d = dataset_dicts[i]
    im = cv2.imread(d["file_name"])
    output = predictor(im)
    bbox = [ float(d['annotations'][0]['bbox'][i]) for i in range(4)]

    cuda0 = torch.device('cuda:0')
    bboxes_pred = outputs["instances"].pred_boxes
    gt = torch.empty_like(bboxes_pred.tensor)


    for j in range(4):
        gt[0][j] = bbox[j]

    bboxes_gt = structures.Boxes(gt)

    IOUs = structures.pairwise_iou(bboxes_gt, bboxes_pred)
    if IOUs[0][0] > 0.5 :
      print(bboxes_gt, bboxes_pred, IOUs)
      print(output)
      print(d)